# Цель работы:
Разложить картинку по svd для каждого канала RGB в отдельности и потом собрать цвета обратно вместе, получив цветные картинки для разных количеств сингулярных значений.

##  1) Подготовка
Для выполнения работы надо импонтировать Image из PIL, чтобы загрузить фото (функция .open()) и после вновь перервести фотографии из матрицы (функция .fromarray()) и сохранить их (функция .save()). И импонтировать numpy, для работы с матрицами.
Загружаем картинку, добавленную в папку заранее (папка с кодом).
В дальнейшем также понадобятся размеры картинки, поэтому сохраняем их в виде переменных w, h.

In [19]:
from PIL import Image
import numpy as np
img = Image.open("pavlin.jpg")
w = img.size[0]
h = img.size[1]

## 2) SVD разложение для каждого цвета
Переводим картинку в матрицу (x). Используя срезы, выделяем матрицы отвечающих за все три цвета (красный - r, зелёный - g, голубой - b).
Полученные матрицы раскладываем используя функцию numpy - linalg.svd(). Full_matrices = False - чтобы ненужная часть матрицы не учитывалась.

In [24]:
x = np.array(img, dtype = np.float32)

r, g, b = x[:,:,0], x[:,:,1], x[:,:,2]

U_r, S_r, V_r = np.linalg.svd(r,full_matrices = False) #красный
U_g, S_g, V_g = np.linalg.svd(g,full_matrices = False) #зелённый
U_b, S_b, V_b = np.linalg.svd(b,full_matrices = False) #голубой


## 3) Собираем и сохраняем картинки, используя разные значения
Чтобы наглядно увидеть изменения картинки, используем цикл for от 1 до 100 с шагом 5. 
Каждый цикл для каждого цвета:
 1. получаем матрицу с помощью умножения соответсвующих матриц U, S, V по нужным строкам и столбцам.
 2. для борьбы с переполнением (возникнет при смене типа float32 на uint8), меняем отрицательные значения на 0, а превышающие 255 на 255.
 3. добавляем третье измерение используя функцию reshape и сохранённые ранее значения ширины и длины (w,h)
В самом конце, цикл с помощью функции concatenate по оси 2 чтобы получить матрицу похожую на матрицу исходного изображения. Последней строкой переводим получившуюся матрицу в цветную картинку и сохраняем в формате png.

In [22]:
for i in range (1,100,5):
    red = U_r[:,:i].dot(np.diag(S_r[:i])).dot(V_r[:i,:])
    green = U_g[:,:i].dot(np.diag(S_g[:i])).dot(V_g[:i,:])
    blue = U_b[:,:i].dot(np.diag(S_b[:i])).dot(V_b[:i,:])
    
    red[red < 0] = 0
    red[red > 255] = 255
    green[green < 0] = 0
    green[green > 255] = 255
    blue[blue < 0] = 0
    blue[blue > 255] = 255
    
    red = red.reshape(h, w, 1)
    green = green.reshape(h, w, 1)
    blue = blue.reshape(h, w, 1)
    

    Y_r = np.concatenate((np.concatenate((red, green), axis = 2), blue), axis = 2)            
    Image.fromarray(np.asarray(Y_r,dtype = np.uint8)).save(f'result{i}.png')